In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import io
from termcolor import colored

In [2]:
df=pd.read_csv('BHARTITEST.csv',index_col=[0])

In [3]:
df['DateTime']=pd.to_datetime(df['Date']+" "+df['Time'])
df=df.reindex(columns=[ 'DateTime','Date', 'Time', 'Open', 'High', 'Low', 'Close'])
df.drop(['Date','Time'],axis=1,inplace=True)

In [4]:
df

,DateTime,Open,High,Low,Close
564885,2020-03-01 09:15:59,457.25,457.95,455.00,455.30
564886,2020-03-01 09:16:59,455.05,456.20,455.05,456.20
564887,2020-03-01 09:17:59,456.20,456.20,454.55,454.75
564888,2020-03-01 09:18:59,454.55,455.10,454.55,455.10
564889,2020-03-01 09:19:59,455.30,456.25,455.30,455.65
...,...,...,...,...,...
5737722,2020-01-30 15:25:59,490.05,490.10,489.70,489.95
5737723,2020-01-30 15:26:59,489.95,489.95,489.30,489.35
5737724,2020-01-30 15:27:59,489.45,489.45,488.25,488.25
5737725,2020-01-30 15:28:59,488.35,488.65,488.00,488.00


In [5]:
df['DateTime'].unique()

array(['2020-03-01T09:15:59.000000000', '2020-03-01T09:16:59.000000000',
       '2020-03-01T09:17:59.000000000', ...,
       '2020-01-30T15:27:59.000000000', '2020-01-30T15:28:59.000000000',
       '2020-01-30T15:29:59.000000000'], dtype='datetime64[ns]')

In [6]:
df2=pd.read_csv('BHARTITRAIN.csv',index_col=[0])

In [7]:
df2.reset_index(drop=True,inplace=True)

In [8]:
df2

,DateTime,Open,High,Low,Close
0,2020-01-01 12:45:59,452.05,462.55,451.55,452.40
1,2020-01-01 12:50:59,452.40,462.55,451.55,452.80
2,2020-01-01 12:55:59,452.80,462.55,451.55,452.50
3,2020-01-01 13:00:59,452.50,462.55,451.55,452.75
4,2020-01-01 13:05:59,453.00,462.55,451.55,453.00
...,...,...,...,...,...
95,2020-02-01 15:05:59,457.80,462.55,451.55,458.60
96,2020-02-01 15:10:59,458.60,462.55,451.55,457.70
97,2020-02-01 15:15:59,457.70,462.55,451.55,458.10
98,2020-02-01 15:20:59,458.10,462.55,451.55,458.60


In [9]:
lst=df2['Close'].to_list()

In [10]:
last10=lst[-14:]
last10

[458.7,
 458.0,
 457.95,
 458.0,
 458.1,
 458.15,
 458.4,
 458.75,
 457.8,
 458.6,
 457.7,
 458.1,
 458.6,
 458.5]

In [11]:
long_target=1.25
long_sl=-0.5

short_target=-1.25
short_sl=0.5
df['SMA']=0.0

In [12]:
tradesheet=pd.DataFrame(columns=['EntryTime','ExitTime','TradeType','EntryPrice','ExitPrice','Spread'])

In [13]:
import sys
from datetime import timedelta
from datetime import datetime
hashmap={0:[last10[-1],sum(last10)/len(last10)]}
longt=0
longsl=0
shortt=0
shortsl=0
high=0
low=10000007
trade=0
flag=0
close=455.30
ltarget=-1
starget=-1
sstoploss=-1
lstoploss=-1
data={}
for i,j in enumerate(zip(df.Close,df.SMA,df.High,df.Low,df.DateTime,df.Open)):
    tradcount=0
    res=[*j]
    sma=0
    print("___ DATA ___")
    print('DATETIME:',res[4])
    print("OPEN:",res[5])
    print("HIGH:",res[2])
    print("LOW",res[3])
    print("CLOSE",res[0])
    print("------------------")
    
    #long entry short exit-> green 
    #long exit short entry -> red
  
    #df.Close 0 ,df.SMA10 1,df.High 2,df.Low 3 ,df.DateTime,df.Open
    
    if res[4].minute%5==0:
        dx={}
        last10.pop(0)
        last10.append(res[0])
        sma=sum(last10)/len(last10)
        print(last10,res[0])
        print("SMA-->",sma)
        df.iat[i, df.columns.get_loc('SMA')]=sma
        dx={'DateTime':res[4],
             'Open':res[5]}
        high=res[2]
        low=res[3]
        close=res[0]
    
    elif res[4].minute%5!=0:
        last10[-1]=res[0]
        sma=sum(last10)/len(last10)
        #print("n",res[3],last10)
        print(last10,res[0])
        print("SMA-->",sma)
        df.iat[i, df.columns.get_loc('SMA')]=sma 
        
        high=max(high,res[2])
        low=min(low,res[3])
        close=res[0]
        if (df['DateTime'].iloc[i]+timedelta(minutes=1)).minute%5==0:
            #print(df['DateTime'].iloc[i])
            dx['High']=high
            dx['Low']=low
            dx['Close']=close
            print()
            print("Pushing Data")
            print(dx)
            df2=df2.append(dx,ignore_index=True)
      
    if trade==0 and res[0]>sma and hashmap[i][0]<hashmap[i][1]:
        print(colored("LONG ENTRY CONDITION SATISFIED !",'green'))
        trade=1
        tradcount+=1
        flag=1
        print("1. Close :{} > SMA: {}".format(res[0],sma))
        print("2. PrevClose :{} < PrevSMA :{}".format(hashmap[i][0],hashmap[i][1]))
        print()
        print("BUY BHARTIAIRTEL AT ",res[4],"IN Rs",res[0])
        ltarget=res[0]+res[0]*long_target/100
        lstoploss=res[0]+res[0]*long_sl/100
        
        data={'EntryTime':df['DateTime'].iloc[i],'ExitTime':None,'TradeType':'LONG','EntryPrice':df['Close'].iloc[i],'ExitPrice':None,'Spread':None}
        
        print('Target->',ltarget,'Stoploss->',lstoploss)
        
        print()
    if tradcount==1:
        tradcount+=1
        continue
        
        
    if res[2]>ltarget and flag==1:
        print(colored('Target Achieved- Long Exit','red'))
        data['ExitTime']=res[4]
        data['ExitPrice']=res[2]
        data['Spread']=data['ExitPrice']-data['EntryPrice']
        print(data)
        tradesheet=tradesheet.append(data,ignore_index=True)
        data={}
        trade=0
        flag=0
        tradcount=0
    elif res[3]<lstoploss and flag==1:
        print(colored("Stoploss Achieved- Long Exit",'red'))
        data['ExitTime']=res[4]
        data['ExitPrice']=res[3]
        data['Spread']=data['ExitPrice']-data['EntryPrice']
        print(data)
        tradesheet=tradesheet.append(data,ignore_index=True)
        data={}
        trade=0
        flag=0
        tradcount=0


    if trade==0 and res[0]<sma and hashmap[i][0]>hashmap[i][1]:
        trade=1
        tradcount+=1
        flag=2
        print(colored("SHORT ENTRY CONDITION SATISFIED !",'red'))
        print("1. Close :{} < SMA: {}".format(res[0],sma))
        print("2. PrevClose :{} > PrevSMA :{}".format(hashmap[i][0],hashmap[i][1]))
        print()
        print("SELL BHARTIAIRTEL AT ",res[4],"IN Rs",res[0])
        starget=res[0]+res[0]*short_target/100
        sstoploss=res[0]+res[0]*short_sl/100
        data={'EntryTime':df['DateTime'].iloc[i],'ExitTime':0,'TradeType':'SHORT','EntryPrice':df['Close'].iloc[i],'ExitPrice':0,'Spread':0}
        
        print('Target->',starget,'Stoploss->',sstoploss)
        print()
    if tradcount==1:
        tradcount+=1
        continue

    if res[3]<starget and flag==2:
        print(colored('Short_Target Achieved','green'))
        data['ExitTime']=res[4]
        data['ExitPrice']=res[3]
        data['Spread']=data['EntryPrice']-data['ExitPrice']
        print(data)
        tradesheet=tradesheet.append(data,ignore_index=True)
        data={}
        trade=0
        flag=0
    elif res[2]>sstoploss and flag==2:
        print(colored("Short_Stoploss Achieved",'red'))
        data['ExitTime']=res[4]
        data['ExitPrice']=res[2]
        data['Spread']=data['EntryPrice']-data['ExitPrice']
        
        tradesheet=tradesheet.append(data,ignore_index=True)
        
        data={}
        trade=0
        flag=0

        
    #print(trade,flag,tradcount)
    hashmap[i+1]=[res[0],sma]
    
            
        

___ DATA ___
DATETIME: 2020-03-01 09:15:59
OPEN: 457.25
HIGH: 457.95
LOW 455.0
CLOSE 455.3
------------------
[458.0, 457.95, 458.0, 458.1, 458.15, 458.4, 458.75, 457.8, 458.6, 457.7, 458.1, 458.6, 458.5, 455.3] 455.3
SMA--> 457.9964285714287
SHORT ENTRY CONDITION SATISFIED !
1. Close :455.3 < SMA: 457.9964285714287
2. PrevClose :458.5 > PrevSMA :458.2392857142858

SELL BHARTIAIRTEL AT  2020-03-01 09:15:59 IN Rs 455.3
Target-> 449.60875 Stoploss-> 457.5765

___ DATA ___
DATETIME: 2020-03-01 09:16:59
OPEN: 455.05
HIGH: 456.2
LOW 455.05
CLOSE 456.2
------------------
[458.0, 457.95, 458.0, 458.1, 458.15, 458.4, 458.75, 457.8, 458.6, 457.7, 458.1, 458.6, 458.5, 456.2] 456.2
SMA--> 458.06071428571437
___ DATA ___
DATETIME: 2020-03-01 09:17:59
OPEN: 456.2
HIGH: 456.2
LOW 454.55
CLOSE 454.75
------------------
[458.0, 457.95, 458.0, 458.1, 458.15, 458.4, 458.75, 457.8, 458.6, 457.7, 458.1, 458.6, 458.5, 454.75] 454.75
SMA--> 457.95714285714297
___ DATA ___
DATETIME: 2020-03-01 09:18:59
OPEN:

___ DATA ___
DATETIME: 2020-03-01 10:15:59
OPEN: 459.35
HIGH: 459.7
LOW 459.35
CLOSE 459.7
------------------
[458.5, 455.65, 456.05, 455.85, 457.2, 457.15, 457.45, 457.6, 457.1, 458.4, 459.5, 460.1, 459.3, 459.7] 459.7
SMA--> 457.825
___ DATA ___
DATETIME: 2020-03-01 10:16:59
OPEN: 459.45
HIGH: 459.45
LOW 459.35
CLOSE 459.35
------------------
[458.5, 455.65, 456.05, 455.85, 457.2, 457.15, 457.45, 457.6, 457.1, 458.4, 459.5, 460.1, 459.3, 459.35] 459.35
SMA--> 457.80000000000007
___ DATA ___
DATETIME: 2020-03-01 10:17:59
OPEN: 459.35
HIGH: 459.45
LOW 459.3
CLOSE 459.45
------------------
[458.5, 455.65, 456.05, 455.85, 457.2, 457.15, 457.45, 457.6, 457.1, 458.4, 459.5, 460.1, 459.3, 459.45] 459.45
SMA--> 457.8071428571429
___ DATA ___
DATETIME: 2020-03-01 10:18:59
OPEN: 459.45
HIGH: 459.45
LOW 459.2
CLOSE 459.2
------------------
[458.5, 455.65, 456.05, 455.85, 457.2, 457.15, 457.45, 457.6, 457.1, 458.4, 459.5, 460.1, 459.3, 459.2] 459.2
SMA--> 457.7892857142857
___ DATA ___
DATETIME:

___ DATA ___
DATETIME: 2020-03-01 13:10:59
OPEN: 454.05
HIGH: 454.2
LOW 454.05
CLOSE 454.1
------------------
[455.1, 454.95, 454.35, 454.1, 454.85, 454.4, 454.5, 454.65, 454.5, 454.6, 454.0, 453.9, 454.05, 454.1] 454.1
SMA--> 454.4321428571429
___ DATA ___
DATETIME: 2020-03-01 13:11:59
OPEN: 454.15
HIGH: 454.45
LOW 454.15
CLOSE 454.4
------------------
[455.1, 454.95, 454.35, 454.1, 454.85, 454.4, 454.5, 454.65, 454.5, 454.6, 454.0, 453.9, 454.05, 454.4] 454.4
SMA--> 454.45357142857137
___ DATA ___
DATETIME: 2020-03-01 13:12:59
OPEN: 454.4
HIGH: 454.4
LOW 454.2
CLOSE 454.2
------------------
[455.1, 454.95, 454.35, 454.1, 454.85, 454.4, 454.5, 454.65, 454.5, 454.6, 454.0, 453.9, 454.05, 454.2] 454.2
SMA--> 454.4392857142857
___ DATA ___
DATETIME: 2020-03-01 13:13:59
OPEN: 454.2
HIGH: 454.35
LOW 454.2
CLOSE 454.25
------------------
[455.1, 454.95, 454.35, 454.1, 454.85, 454.4, 454.5, 454.65, 454.5, 454.6, 454.0, 453.9, 454.05, 454.25] 454.25
SMA--> 454.4428571428571
___ DATA ___
DATET

___ DATA ___
DATETIME: 2020-06-01 09:52:59
OPEN: 451.7
HIGH: 451.7
LOW 450.25
CLOSE 450.7
------------------
[455.55, 456.45, 456.2, 456.1, 458.45, 458.0, 458.1, 455.35, 454.1, 455.0, 454.75, 453.2, 452.25, 450.7] 450.7
SMA--> 455.3
___ DATA ___
DATETIME: 2020-06-01 09:53:59
OPEN: 450.65
HIGH: 450.75
LOW 450.1
CLOSE 450.75
------------------
[455.55, 456.45, 456.2, 456.1, 458.45, 458.0, 458.1, 455.35, 454.1, 455.0, 454.75, 453.2, 452.25, 450.75] 450.75
SMA--> 455.30357142857144
___ DATA ___
DATETIME: 2020-06-01 09:54:59
OPEN: 450.6
HIGH: 451.2
LOW 450.5
CLOSE 451.0
------------------
[455.55, 456.45, 456.2, 456.1, 458.45, 458.0, 458.1, 455.35, 454.1, 455.0, 454.75, 453.2, 452.25, 451.0] 451.0
SMA--> 455.32142857142856

Pushing Data
{'DateTime': Timestamp('2020-06-01 09:50:59'), 'Open': 452.3, 'High': 452.35, 'Low': 450.1, 'Close': 451.0}
___ DATA ___
DATETIME: 2020-06-01 09:55:59
OPEN: 451.0
HIGH: 451.7
LOW 451.0
CLOSE 451.15
------------------
[456.45, 456.2, 456.1, 458.45, 458.0, 458

___ DATA ___
DATETIME: 2020-06-01 12:35:59
OPEN: 448.35
HIGH: 448.6
LOW 448.35
CLOSE 448.6
------------------
[451.0, 451.95, 450.95, 450.8, 449.8, 449.95, 449.9, 448.25, 448.7, 449.3, 448.8, 447.25, 448.35, 448.6] 448.6
SMA--> 449.54285714285714
___ DATA ___
DATETIME: 2020-06-01 12:36:59
OPEN: 448.6
HIGH: 448.8
LOW 448.6
CLOSE 448.6
------------------
[451.0, 451.95, 450.95, 450.8, 449.8, 449.95, 449.9, 448.25, 448.7, 449.3, 448.8, 447.25, 448.35, 448.6] 448.6
SMA--> 449.54285714285714
___ DATA ___
DATETIME: 2020-06-01 12:37:59
OPEN: 448.6
HIGH: 448.65
LOW 448.4
CLOSE 448.4
------------------
[451.0, 451.95, 450.95, 450.8, 449.8, 449.95, 449.9, 448.25, 448.7, 449.3, 448.8, 447.25, 448.35, 448.4] 448.4
SMA--> 449.5285714285714
___ DATA ___
DATETIME: 2020-06-01 12:38:59
OPEN: 448.4
HIGH: 448.5
LOW 448.4
CLOSE 448.4
------------------
[451.0, 451.95, 450.95, 450.8, 449.8, 449.95, 449.9, 448.25, 448.7, 449.3, 448.8, 447.25, 448.35, 448.4] 448.4
SMA--> 449.5285714285714
___ DATA ___
DATETI

...

___ DATA ___
DATETIME: 2020-01-27 14:55:59
OPEN: 516.15
HIGH: 516.75
LOW 516.05
CLOSE 516.5
------------------
[518.0, 517.95, 516.7, 517.6, 516.8, 517.15, 517.15, 516.85, 516.5, 516.1, 516.65, 515.25, 515.9, 516.5] 516.5
SMA--> 516.7928571428572
___ DATA ___
DATETIME: 2020-01-27 14:56:59
OPEN: 516.5
HIGH: 516.55
LOW 516.2
CLOSE 516.5
------------------
[518.0, 517.95, 516.7, 517.6, 516.8, 517.15, 517.15, 516.85, 516.5, 516.1, 516.65, 515.25, 515.9, 516.5] 516.5
SMA--> 516.7928571428572
___ DATA ___
DATETIME: 2020-01-27 14:57:59
OPEN: 516.55
HIGH: 516.7
LOW 516.3
CLOSE 516.5
------------------
[518.0, 517.95, 516.7, 517.6, 516.8, 517.15, 517.15, 516.85, 516.5, 516.1, 516.65, 515.25, 515.9, 516.5] 516.5
SMA--> 516.7928571428572
___ DATA ___
DATETIME: 2020-01-27 14:58:59
OPEN: 516.45
HIGH: 516.45
LOW 515.75
CLOSE 515.75
------------------
[518.0, 517.95, 516.7, 517.6, 516.8, 517.15, 517.15, 516.85, 516.5, 516.1, 516.65, 515.25, 515.9, 515.75] 515.75
SMA--> 516.7392857142858
___ DATA ___


___ DATA ___
DATETIME: 2020-01-29 11:30:59
OPEN: 493.55
HIGH: 493.8
LOW 493.45
CLOSE 493.8
------------------
[495.9, 495.85, 495.3, 494.9, 494.6, 494.65, 494.75, 495.3, 495.65, 495.35, 494.25, 493.35, 493.4, 493.8] 493.8
SMA--> 494.7892857142857
___ DATA ___
DATETIME: 2020-01-29 11:31:59
OPEN: 493.8
HIGH: 493.8
LOW 493.45
CLOSE 493.45
------------------
[495.9, 495.85, 495.3, 494.9, 494.6, 494.65, 494.75, 495.3, 495.65, 495.35, 494.25, 493.35, 493.4, 493.45] 493.45
SMA--> 494.7642857142857
___ DATA ___
DATETIME: 2020-01-29 11:32:59
OPEN: 493.45
HIGH: 493.6
LOW 493.3
CLOSE 493.3
------------------
[495.9, 495.85, 495.3, 494.9, 494.6, 494.65, 494.75, 495.3, 495.65, 495.35, 494.25, 493.35, 493.4, 493.3] 493.3
SMA--> 494.75357142857143
___ DATA ___
DATETIME: 2020-01-29 11:33:59
OPEN: 493.3
HIGH: 493.9
LOW 493.3
CLOSE 493.65
------------------
[495.9, 495.85, 495.3, 494.9, 494.6, 494.65, 494.75, 495.3, 495.65, 495.35, 494.25, 493.35, 493.4, 493.65] 493.65
SMA--> 494.7785714285714
___ DATA 

___ DATA ___
DATETIME: 2020-01-29 14:20:59
OPEN: 491.05
HIGH: 491.25
LOW 491.05
CLOSE 491.25
------------------
[489.8, 489.35, 490.3, 490.6, 490.1, 490.7, 491.7, 492.1, 491.35, 491.5, 490.15, 490.75, 491.05, 491.25] 491.25
SMA--> 490.7642857142857
___ DATA ___
DATETIME: 2020-01-29 14:21:59
OPEN: 491.25
HIGH: 491.45
LOW 491.25
CLOSE 491.45
------------------
[489.8, 489.35, 490.3, 490.6, 490.1, 490.7, 491.7, 492.1, 491.35, 491.5, 490.15, 490.75, 491.05, 491.45] 491.45
SMA--> 490.7785714285714
___ DATA ___
DATETIME: 2020-01-29 14:22:59
OPEN: 491.45
HIGH: 491.45
LOW 491.3
CLOSE 491.4
------------------
[489.8, 489.35, 490.3, 490.6, 490.1, 490.7, 491.7, 492.1, 491.35, 491.5, 490.15, 490.75, 491.05, 491.4] 491.4
SMA--> 490.775
___ DATA ___
DATETIME: 2020-01-29 14:23:59
OPEN: 491.4
HIGH: 491.4
LOW 491.2
CLOSE 491.2
------------------
[489.8, 489.35, 490.3, 490.6, 490.1, 490.7, 491.7, 492.1, 491.35, 491.5, 490.15, 490.75, 491.05, 491.2] 491.2
SMA--> 490.76071428571424
___ DATA ___
DATETIME: 

___ DATA ___
DATETIME: 2020-01-30 10:55:59
OPEN: 484.25
HIGH: 484.4
LOW 483.4
CLOSE 484.0
------------------
[484.5, 483.35, 481.5, 482.1, 481.95, 481.65, 482.2, 482.1, 481.7, 481.9, 483.4, 482.4, 484.25, 484.0] 484.0
SMA--> 482.642857142857
___ DATA ___
DATETIME: 2020-01-30 10:56:59
OPEN: 484.0
HIGH: 484.0
LOW 483.55
CLOSE 483.55
------------------
[484.5, 483.35, 481.5, 482.1, 481.95, 481.65, 482.2, 482.1, 481.7, 481.9, 483.4, 482.4, 484.25, 483.55] 483.55
SMA--> 482.61071428571415
___ DATA ___
DATETIME: 2020-01-30 10:57:59
OPEN: 483.55
HIGH: 483.55
LOW 482.7
CLOSE 482.7
------------------
[484.5, 483.35, 481.5, 482.1, 481.95, 481.65, 482.2, 482.1, 481.7, 481.9, 483.4, 482.4, 484.25, 482.7] 482.7
SMA--> 482.54999999999984
___ DATA ___
DATETIME: 2020-01-30 10:58:59
OPEN: 482.7
HIGH: 482.8
LOW 482.6
CLOSE 482.65
------------------
[484.5, 483.35, 481.5, 482.1, 481.95, 481.65, 482.2, 482.1, 481.7, 481.9, 483.4, 482.4, 484.25, 482.65] 482.65
SMA--> 482.5464285714284
___ DATA ___
DATETIME

___ DATA ___
DATETIME: 2020-01-30 13:40:59
OPEN: 486.6
HIGH: 486.6
LOW 486.4
CLOSE 486.45
------------------
[486.5, 486.65, 486.55, 486.3, 486.45, 484.5, 484.75, 484.15, 484.95, 485.1, 485.4, 485.65, 486.7, 486.45] 486.45
SMA--> 485.72142857142853
___ DATA ___
DATETIME: 2020-01-30 13:41:59
OPEN: 486.45
HIGH: 486.65
LOW 486.45
CLOSE 486.45
------------------
[486.5, 486.65, 486.55, 486.3, 486.45, 484.5, 484.75, 484.15, 484.95, 485.1, 485.4, 485.65, 486.7, 486.45] 486.45
SMA--> 485.72142857142853
___ DATA ___
DATETIME: 2020-01-30 13:42:59
OPEN: 486.45
HIGH: 486.65
LOW 486.35
CLOSE 486.35
------------------
[486.5, 486.65, 486.55, 486.3, 486.45, 484.5, 484.75, 484.15, 484.95, 485.1, 485.4, 485.65, 486.7, 486.35] 486.35
SMA--> 485.7142857142857
___ DATA ___
DATETIME: 2020-01-30 13:43:59
OPEN: 486.35
HIGH: 486.35
LOW 485.4
CLOSE 485.4
------------------
[486.5, 486.65, 486.55, 486.3, 486.45, 484.5, 484.75, 484.15, 484.95, 485.1, 485.4, 485.65, 486.7, 485.4] 485.4
SMA--> 485.64642857142854


In [14]:
tradesheet

,EntryTime,ExitTime,TradeType,EntryPrice,ExitPrice,Spread
0,2020-03-01 09:15:59,2020-03-01 09:44:59,SHORT,455.3,457.65,-2.35
1,2020-03-01 09:49:59,2020-03-01 11:45:59,LONG,457.6,455.2,-2.4
2,2020-03-01 12:47:59,2020-03-01 14:25:59,LONG,454.95,452.3,-2.65
3,2020-03-01 14:36:59,2020-06-01 09:51:59,LONG,454.0,451.55,-2.45
4,2020-06-01 10:40:59,2020-06-01 11:01:59,LONG,452.2,449.5,-2.7
5,2020-06-01 11:07:59,2020-06-01 11:48:59,LONG,451.8,449.05,-2.75
6,2020-06-01 13:00:59,2020-07-01 09:17:59,LONG,448.85,454.65,5.8
7,2020-07-01 10:15:59,2020-07-01 11:28:59,SHORT,454.3,448.4,5.9
8,2020-07-01 12:46:59,2020-07-01 13:24:59,LONG,448.45,446.15,-2.3
9,2020-07-01 13:42:59,2020-07-01 15:26:59,LONG,447.8,445.5,-2.3


In [15]:
df1=tradesheet.copy()

In [16]:
qty = 100
initial_cap=100000
pnl = df1["Spread"].tolist()
df1["pnl"]=df1["Spread"] * qty
df1["Balance"] = 0
df1["cumm_ret"] = 0
df1["strategy_return"] = 0

for i in range(len(df1)):
    if(i == 0): 
        df1["Balance"].iloc[i] = df1["pnl"].iloc[i] + initial_cap
    else:
        df1["Balance"].iloc[i] = df1["pnl"].iloc[i] + df1["Balance"].iloc[i-1]
        
for i in range(len(df1)):
    if(i == 0):
        df1["strategy_return"].iloc[i] = (df1["Balance"].iloc[i] - initial_cap)/initial_cap
    else:
        df1["strategy_return"].iloc[i] = (df1["Balance"].iloc[i] - df1["Balance"].iloc[i-1])/df1["Balance"].iloc[i-1]

for i in range(len(df1)):
    df1["cumm_ret"].iloc[i] = (df1["Balance"].iloc[i] - initial_cap)/initial_cap

In [17]:
df1

,EntryTime,ExitTime,TradeType,EntryPrice,ExitPrice,Spread,pnl,Balance,cumm_ret,strategy_return
0,2020-03-01 09:15:59,2020-03-01 09:44:59,SHORT,455.3,457.65,-2.35,-235.0,99765.0,-0.00235,-0.002350
1,2020-03-01 09:49:59,2020-03-01 11:45:59,LONG,457.6,455.2,-2.4,-240.0,99525.0,-0.00475,-0.002406
2,2020-03-01 12:47:59,2020-03-01 14:25:59,LONG,454.95,452.3,-2.65,-265.0,99260.0,-0.00740,-0.002663
3,2020-03-01 14:36:59,2020-06-01 09:51:59,LONG,454.0,451.55,-2.45,-245.0,99015.0,-0.00985,-0.002468
4,2020-06-01 10:40:59,2020-06-01 11:01:59,LONG,452.2,449.5,-2.7,-270.0,98745.0,-0.01255,-0.002727
5,2020-06-01 11:07:59,2020-06-01 11:48:59,LONG,451.8,449.05,-2.75,-275.0,98470.0,-0.01530,-0.002785
6,2020-06-01 13:00:59,2020-07-01 09:17:59,LONG,448.85,454.65,5.8,580.0,99050.0,-0.00950,0.005890
7,2020-07-01 10:15:59,2020-07-01 11:28:59,SHORT,454.3,448.4,5.9,590.0,99640.0,-0.00360,0.005957
8,2020-07-01 12:46:59,2020-07-01 13:24:59,LONG,448.45,446.15,-2.3,-230.0,99410.0,-0.00590,-0.002308
9,2020-07-01 13:42:59,2020-07-01 15:26:59,LONG,447.8,445.5,-2.3,-230.0,99180.0,-0.00820,-0.002314


In [18]:
pos = 0
neg = 0
for i in range(len(pnl)):
    if(pnl[i]>0):
        pos = pos + 1
    else:
        neg = neg + 1
pnl = sum(pnl) * qty
final_cap=initial_cap+pnl
final_cap

93425.0

In [19]:
import numpy as np

In [20]:
risk_free_rate = 0.04/20

sharpe = np.sqrt(20)*(np.mean(df1.strategy_return) -(risk_free_rate))/np.std(df1.strategy_return)

In [21]:
sharpe

-3.2920788946440167

In [22]:
new_cum_strategy_returns = (df1['strategy_return']+1).cumprod()
# Total number of trading days
days = len(new_cum_strategy_returns)

# Calculate compounded annual growth rate
cagr = (new_cum_strategy_returns.iloc[-1]**(20/days)-1)*100
print('The CAGR of strategy is %.2f' % cagr + '%')

The CAGR of strategy is -2.85%


In [23]:
def calc_drawdown(cum_rets):
    # Calculate the running maximum
    running_max = np.maximum.accumulate(cum_rets.dropna())

    # Ensure the value never drops below 1
    running_max[running_max < 1] = 1

    # Calculate the percentage drawdown
    drawdown = (cum_rets)/running_max - 1

    # Calculate maximum drawdown
    max_dd = drawdown.min()*100

    return drawdown, max_dd

drawdown, max_dd = calc_drawdown(new_cum_strategy_returns)

In [24]:
drawdown

0    -0.00235
1    -0.00475
2    -0.00740
3    -0.00985
4    -0.01255
5    -0.01530
6    -0.00950
7    -0.00360
8    -0.00590
9    -0.00820
10   -0.01105
11   -0.01390
12   -0.01710
13   -0.01945
14   -0.02250
15   -0.02490
16   -0.02735
17   -0.02985
18   -0.03265
19   -0.03620
20   -0.03860
21   -0.03235
22   -0.01360
23   -0.01825
24   -0.02400
25   -0.02670
26   -0.02960
27   -0.03240
28   -0.03500
29   -0.03770
30   -0.04040
31   -0.04380
32   -0.04705
33   -0.05760
34   -0.06040
35   -0.06335
36   -0.05680
37   -0.05970
38   -0.06250
39   -0.05580
40   -0.05845
41   -0.06110
42   -0.06370
43   -0.06660
44   -0.06920
45   -0.06250
46   -0.06575
Name: strategy_return, dtype: float64

In [25]:
max_dd

-6.920000000000027

In [26]:
from tabulate import tabulate

stats = tabulate(
        [
            ['Initial Capital', initial_cap],
            ['Ending Capital', round(final_cap, 2)],
            ['Num of positive trades', pos],
            ['Num of negative trades', neg],
            ['Net profit', round(pnl, 2)],
            ['Sharpe Ratio', str(round(sharpe, 2)) + " %"],
            ['CAGR', str(round(cagr, 2)) + " %"],
            ['MDD', str(round(max_dd, 2)) + " %"],
            ['CAGR/MDD', str(round(abs(cagr / max_dd), 2)) + " %"],
        ],
        headers=['Stats Name','Short Straddle'], tablefmt='orgtbl')

print(stats)

| Stats Name             | Short Straddle   |
|------------------------+------------------|
| Initial Capital        | 100000           |
| Ending Capital         | 93425.0          |
| Num of positive trades | 7                |
| Num of negative trades | 40               |
| Net profit             | -6575.0          |
| Sharpe Ratio           | -3.29 %          |
| CAGR                   | -2.85 %          |
| MDD                    | -6.92 %          |
| CAGR/MDD               | 0.41 %           |


In [27]:
df2

,DateTime,Open,High,Low,Close
0,2020-01-01 12:45:59,452.05,462.55,451.55,452.40
1,2020-01-01 12:50:59,452.40,462.55,451.55,452.80
2,2020-01-01 12:55:59,452.80,462.55,451.55,452.50
3,2020-01-01 13:00:59,452.50,462.55,451.55,452.75
4,2020-01-01 13:05:59,453.00,462.55,451.55,453.00
...,...,...,...,...,...
1289,2020-01-30 15:05:59,490.00,490.65,489.95,490.20
1290,2020-01-30 15:10:59,490.20,490.45,489.25,489.90
1291,2020-01-30 15:15:59,489.85,490.25,489.50,490.00
1292,2020-01-30 15:20:59,490.00,490.45,489.65,489.95


In [28]:
df2.to_csv('5minCandle1-30.csv')